# Ranking for property

In [66]:
import sys
sys.path.append("../repository/")
import database as db
import polars as pl

In [67]:
conn_wgi = db.DatabaseConnection("world-governance-index").connection
conn_ti = db.DatabaseConnection("transformation-index").connection
conn_ef = db.DatabaseConnection("economic-freedom").connection
conn_db = db.DatabaseConnection("doing-bussiness").connection




In [68]:
db_rank_registering_property = pl.read_database(
        query="""
        SELECT country_id, rank_registering_property
        FROM "freedom"
        WHERE YEAR = 2020
        ORDER BY rank_registering_property
        """,
        connection=conn_db,
    )

ef_property_rights = pl.read_database(
        query="""
        SELECT country_id, real_property, property_rights
        FROM "freedom"
        WHERE YEAR = 2020
        ORDER BY country_id
        """,
        connection=conn_ef,
    )

ti_property_rights = pl.read_database(
        query="""
        SELECT country_id, property_rights, private_property
        FROM "freedom"
        WHERE YEAR = 2020
        ORDER BY country_id
        """,
        connection=conn_ti,
    )

ef_weights = [1.5, 1.0]
ti_weights = [0.5, 0.5]
db_weight = 3.5



In [69]:
ef_property_rights = ef_property_rights.with_columns(
    pl.fold(
        acc=pl.lit(0),
        function=lambda c1, c2: c1 + c2,
        exprs=[
            pl.col(col) * pl.lit(wgt)
            for col, wgt in zip(ef_property_rights.columns[1:], ef_weights)
        ],
    ).alias("score")
).sort(by="score", descending=True)

ef_property_rights = ef_property_rights.with_columns(
    ef_rank=ef_property_rights["score"].rank("ordinal", descending=True)
).select(pl.col(["country_id", "ef_rank"]))


ti_property_rights = ti_property_rights.with_columns(
    pl.sum_horizontal(pl.exclude("country_id")).rank("ordinal").alias("ti_rank")
).select(pl.col(["country_id", "ti_rank"]))


db_rank_registering_property = db_rank_registering_property.rename(
    {"rank_registering_property": "db_rank"}
)


df = db_rank_registering_property.join(
    ef_property_rights, how="left", on="country_id"
).join(ti_property_rights, how="left", on="country_id")

In [70]:
df.sort(by=["ef_rank", "db_rank", "ti_rank"], descending=False, nulls_last=True)

country_id,db_rank,ef_rank,ti_rank
i64,i64,u32,u32
215,18,1,null
158,2,2,null
60,11,3,null
69,6,4,117
200,21,5,113
166,15,6,null
195,19,7,68
233,10,8,75
44,63,9,116
